In [3]:
import pandas as pd
import numpy as np

In [4]:
titanic_train = pd.read_csv('/Users/leo/Documents/python code/code_snippets/titanic/train.csv')
titanic_test = pd.read_csv('/Users/leo/Documents/python code/code_snippets/titanic/test.csv')

In [5]:
titanic_train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [6]:
titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


missing data
- Remove Age and Cabin: too many missing
- Remove rows where Embarked missing (just 2)

categorical data
- apply transformations
Pclass - the numbers actually do have a meaning. maybe can use it in another way?
Sex
Embarked


numeric data
- apply transformations
SibSp - number of siblings/spouses
Parch - number parents/children
Age
Fare

Feature engineering



In [65]:
# select certain features
X_train = titanic_train.loc[:,['Pclass','Sex','SibSp','Parch', 'Fare', 'Embarked']]
y_train = titanic_train.Survived
print(X_train.shape)
print(y_train.shape)

(891, 6)
(891,)


In [67]:
# remove missing rows from Embarked
index_na_train = X_train[X_train.Embarked.isna()].index
# drop from train
X_train.drop(index = index_na_train, inplace = True)
# drop from test
y_train.drop(index = index_na_train, inplace = True)
# check for NAs
X_train.isna().sum()
print(X_train.shape)
print(y_train.shape)

(889, 6)
(889,)


In [10]:
# convert Pclass to string
X_train['Pclass'] = X_train.Pclass.apply(str)

In [11]:
# transform categorical vars to dummy
titanic_train_cat = X_train.select_dtypes(include = 'object').columns
titanic_train_cat

Index(['Pclass', 'Sex', 'Embarked'], dtype='object')

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('cat',OneHotEncoder(),titanic_train_cat)
], remainder = 'passthrough')

X_train_prep = full_pipeline.fit_transform(X_train)
X_train_prep

array([[ 0.    ,  0.    ,  1.    , ...,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  0.    ,  0.    , ...,  1.    ,  0.    , 71.2833],
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 30.    ],
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  7.75  ]])

In [13]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier()

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(rnd_clf, X_train_prep, y_train, cv = 3, scoring = "accuracy")

array([0.72390572, 0.80067568, 0.80405405])

Get confusion matrix

In [15]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(rnd_clf, X_train_prep, y_train, cv = 3)

y_train_pred[:5]

array([0, 1, 0, 1, 0])

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_pred)

array([[467,  82],
       [103, 237]])

In [17]:
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_train, y_train_pred))
print(recall_score(y_train, y_train_pred))

0.7429467084639498
0.6970588235294117


#### Submission

In [18]:
titanic_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [53]:
# select certain features
# need to have PassengerId here for the submission
X_test = titanic_test.loc[:,['PassengerId','Pclass','Sex','SibSp','Parch', 'Fare', 'Embarked']]
print(X_test.shape)

(418, 7)


In [73]:
# dont think I need to remove rows with missing values in the test data
# remove missing rows from Embarked
# index_na_train = X_test[(X_test.Embarked.isna())| (X_test.Fare.isna())].index
# drop from test
# X_test.drop(index = index_na_train, inplace = True)
# check for NAs
X_test.isna().sum()
print(X_test.shape)

(418, 7)


In [71]:
# there is a missing value on Fare for the test set, just to make sure I can submit the result, i will set that value to zero
median = X_test['Fare'].median()
X_test['Fare'].fillna(median, inplace = True)

In [74]:
# convert Pclass to string
X_test['Pclass'] = X_test.Pclass.apply(str)

In [75]:
# transform categorical vars to dummy
titanic_test_cat = X_test.select_dtypes(include = 'object').columns
titanic_test_cat

Index(['Pclass', 'Sex', 'Embarked'], dtype='object')

In [76]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('cat',OneHotEncoder(),titanic_test_cat)
], remainder = 'passthrough')

X_test_prep = full_pipeline.fit_transform(X_test.loc[:,X_test.columns !='PassengerId'])
X_test_prep

array([[ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  7.8292],
       [ 0.    ,  0.    ,  1.    , ...,  1.    ,  0.    ,  7.    ],
       [ 0.    ,  1.    ,  0.    , ...,  0.    ,  0.    ,  9.6875],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  7.25  ],
       [ 0.    ,  0.    ,  1.    , ...,  0.    ,  0.    ,  8.05  ],
       [ 0.    ,  0.    ,  1.    , ...,  1.    ,  1.    , 22.3583]])

In [77]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier()

In [78]:
rnd_clf.fit(X_train_prep, y_train)

y_pred = rnd_clf.predict(X_test_prep)
y_pred[:4]

array([0, 1, 0, 0])

In [79]:
print(X_test.shape)
print(y_pred.shape)

(418, 7)
(418,)


In [80]:
# pd.Series(y_pred).combine(X_test.PassengerId)
submission_190921 = pd.concat([X_test.PassengerId, pd.Series(y_pred)], axis=1)
submission_190921.columns = ['PassengerId','Survived']
submission_190921

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [82]:
submission_190921.to_csv('submission_190921.csv', index = False)

In [83]:
submission_190921.isna().sum()

PassengerId    0
Survived       0
dtype: int64

In [81]:
submission_190921.dtypes

PassengerId    int64
Survived       int64
dtype: object